In [1]:
import os
import cv2
import random
import imutils
import argparse 
from glob import  glob
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier  
from sklearn import svm
import numpy as np

In [2]:
dataset_path = 'train_data/data'
dir_names = [path.split('/')[2] for path in glob(f'{dataset_path}/*')]
target_img_size = (100, 100)
sample_count = 250
print(dir_names)

# random_seed = 42  
# random.seed(random_seed)
# np.random.seed(random_seed)

['t44_hw', '8b_n', 't4', 't2', '1', 'chord', '#', 'b_hw', 't24_hw', '32', 'dot', '8_hw', '#_hw', '2', '8', '2_hw', '4_hw', 'b', 'cross', '4', 'dot_hw', '32b_r', 't24', '16b_n', 'o', '1_hw', '16_hw', '16', '16b_r', 't44', 'clef', '8b_r', '32b_n', 'o_hw']


In [3]:
def extract_raw_pixels(img):
    resized = cv2.resize(img, target_img_size)
    return resized.flatten()

In [4]:
def extract_hsv_histogram(img):
    resized = cv2.resize(img, target_img_size)
    hsv = cv2.cvtColor(resized, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])
    if imutils.is_cv2():
        hist = cv2.normalize(hist)
    else:
        cv2.normalize(hist, hist)
    return hist.flatten()    

In [5]:
def extract_hog_features(img):
    img = cv2.resize(img, target_img_size)
    win_size = (100, 100)
    cell_size = (4, 4)
    block_size_in_cells = (2, 2)
    
    block_size = (block_size_in_cells[1] * cell_size[1], block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 9  # Number of orientation bins
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()
    return h.flatten()

In [6]:
def extract_features(img, feature_set='raw'):
    if feature_set == 'hog':
        return extract_hog_features(img)
    elif feature_set == 'raw':
        return extract_raw_pixels(img)
    else:
        return extract_hsv_histogram(img)

In [7]:
def load_dataset(feature_set='raw'):
    features = []
    labels = []
    for dir_name in dir_names:
        imgs = glob(f'{dataset_path}/{dir_name}/*.png')
        subset = random.sample([i for i in range(len(imgs))], sample_count)
        for i in subset:
            img = cv2.imread(imgs[i])
            labels.append(dir_name)
            features.append(extract_features(img, feature_set))
        
    return features, labels  

In [8]:
random_seed = 42  
random.seed(random_seed)
np.random.seed(random_seed)

classifiers = {
    'SVM': svm.LinearSVC(random_state=random_seed),
    'KNN': KNeighborsClassifier(n_neighbors=7),
    'NN': MLPClassifier(solver='sgd', random_state=random_seed, hidden_layer_sizes=(500,), max_iter=20, verbose=1)
}

In [9]:
def run_experiment(classifier='SVM', feature_set='raw'):
    print('Loading dataset. This will take time ...')
    features, labels = load_dataset(feature_set)
    print('Finished loading dataset.')
    
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=random_seed)
    print('############## Training', model_name, "##############")
    model.fit(train_features, train_labels)
    accuracy = model.score(test_features, test_labels)
    print(model_name, 'accuracy:', accuracy*100, '%')

In [10]:
run_experiment('hog') 
'''
SVM accuracy: 91.52941176470588 %
KNN accuracy: 72.88235294117648 %
NN accuracy: 85.41176470588235 %
'''

Loading dataset. This will take time ...
Finished loading dataset.
############## Training SVM ##############
SVM accuracy: 91.52941176470588 %
############## Training KNN ##############
KNN accuracy: 72.88235294117648 %
############## Training NN ##############
Iteration 1, loss = 3.38381920
Iteration 2, loss = 2.94255237
Iteration 3, loss = 2.53904648
Iteration 4, loss = 2.21735929
Iteration 5, loss = 1.95802995
Iteration 6, loss = 1.74093055
Iteration 7, loss = 1.56065673
Iteration 8, loss = 1.40801272
Iteration 9, loss = 1.28045021
Iteration 10, loss = 1.17315606
Iteration 11, loss = 1.08231356
Iteration 12, loss = 1.00481915
Iteration 13, loss = 0.93811483
Iteration 14, loss = 0.87996494
Iteration 15, loss = 0.82880189
Iteration 16, loss = 0.78413450
Iteration 17, loss = 0.74398625
Iteration 18, loss = 0.70798426
Iteration 19, loss = 0.67540290
Iteration 20, loss = 0.64618737
NN accuracy: 85.41176470588235 %


In [11]:
# run_experiment('hsv')
'''
SVM accuracy: 29.117647058823533 %
KNN accuracy: 56.99999999999999 %
NN accuracy: 7.588235294117647 %
'''

Loading dataset. This will take time ...
Finished loading dataset.
############## Training SVM ##############
SVM accuracy: 29.117647058823533 %
############## Training KNN ##############
KNN accuracy: 56.99999999999999 %
############## Training NN ##############
Iteration 1, loss = 3.52997348
Iteration 2, loss = 3.52867534
Iteration 3, loss = 3.52741392
Iteration 4, loss = 3.52622840
Iteration 5, loss = 3.52506466
Iteration 6, loss = 3.52391650
Iteration 7, loss = 3.52277766
Iteration 8, loss = 3.52168631
Iteration 9, loss = 3.52060229
Iteration 10, loss = 3.51957432
Iteration 11, loss = 3.51859618
Iteration 12, loss = 3.51766670
Iteration 13, loss = 3.51675189
Iteration 14, loss = 3.51585698
Iteration 15, loss = 3.51497376
Iteration 16, loss = 3.51412851
Iteration 17, loss = 3.51329208
Iteration 18, loss = 3.51247892
Iteration 19, loss = 3.51164852
Iteration 20, loss = 3.51081795
NN accuracy: 7.588235294117647 %


In [12]:
# run_experiment('raw')
'''
SVM accuracy: 80.47058823529412 %
KNN accuracy: 83.70588235294117 %
NN accuracy: 9.294117647058824 %
'''

Loading dataset. This will take time ...
Finished loading dataset.
############## Training SVM ##############
SVM accuracy: 80.47058823529412 %
############## Training KNN ##############
KNN accuracy: 83.70588235294117 %
############## Training NN ##############
Iteration 1, loss = 14.04484661
Iteration 2, loss = 5.55339056
Iteration 3, loss = 5.40788561
Iteration 4, loss = 5.36259367
Iteration 5, loss = 5.34016909
Iteration 6, loss = 5.32644265
Iteration 7, loss = 5.29075743
Iteration 8, loss = 5.29009413
Iteration 9, loss = 5.26948073
Iteration 10, loss = 5.26022836
Iteration 11, loss = 5.23375247
Iteration 12, loss = 5.24763582
Iteration 13, loss = 5.24200007
Iteration 14, loss = 5.23015958
Iteration 15, loss = 5.20496847
Iteration 16, loss = 5.18339049
Iteration 17, loss = 5.20256871
Iteration 18, loss = 5.18678817
Iteration 19, loss = 5.19488985
Iteration 20, loss = 5.18156751
NN accuracy: 9.294117647058824 %
